In [1]:
# Importando bibliotecas

import pandas as pd
import os.path
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

**INÍCIO DO SCRIPT PARA ACESSAR O GOOGLE SHEETS**

In [2]:
# Escopo para o acesso - login

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# Tentar ler as credenciais do arquivo token.json
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# Se não houver credenciais válidas, solicite que o usuário o login
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    # Salvar as credenciais para a próxima execução
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [3]:
# Apontar o ID da planilha do Google Sheets
SPREADSHEET_ID = '1X3I2in2Cr6KmnAG4tMWzaucwz-nre84bXaSYXR-sH2g'
# Apontar o intervalo de células que deseja executar a leitura
RANGE_NAME = 'PLANILHA DESCRITIVA INTEGRAL!A:L'
# Ciação da variável 'service' para acessar a API do Google Sheets
service = build('sheets', 'v4', credentials=creds)
# Leitura dos dados da planilha do Google Sheets
result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values')

In [4]:
# Criação de um DataFrame do pandas com os dados lidos
planilha_df = pd.DataFrame(values[1:], columns=values[0])

In [5]:
# Aplicando condivional para separar os processo PAE3 e PAE4
proc_pae3 = [processo for processo in planilha_df['Nº DO PROCESSO'] if processo[0] != 'E']
proc_pae4 = [processo for processo in planilha_df['Nº DO PROCESSO'] if processo[0] == 'E']

In [6]:
# Salvar dataframe no formato excel
planilha_df.to_excel('planilha_google_sheets.xlsx', index=False)

In [7]:
# Separando as informaççoes de ano e processos
'''Esse procedimento é necessário visto que a pagina web do sistema PAE3 de pesquisa possui campos
distintos para tais informações.'''
ano_pae3 = []
processos_pae3 = []

for proc in proc_pae3:
    ano_pae3.append(proc.split('/')[0])
    processos_pae3.append(proc.split('/')[-1])

In [8]:
# Separando as informaççoes de ano e processos
'''É necessário a retirada dos caracteeres 'E-' e não cabe a separação do ano e processo visto 
que a pagina web do sistema PAE4 de aceita apenas o interiro do oprocesso.'''
processos_pae4 = []

for proc in proc_pae4:
    processos_pae4.append(proc.split('-')[1])

In [9]:
print('leitura do google sheets realizada com sucesso!')

leitura do google sheets realizada com sucesso!
